In [24]:
    """
    This code generates a fesibility area for deriving intertidal bathymetry. 
    It computes the area between the the Lowest Astronomical Tide (LAT) and 
    the Mean Seal Level + 10 m, based on a LAT map, and a Bathymetry and 
    elevation GEBCO map. 

        Author: Mario.FuentesMonjaraz@deltares.nl
    """

'\nThis code generates a fesibility area for deriving intertidal bathymetry. \nIt computes the area between the the Lowest Astronomical Tide (LAT) and \nthe Mean Seal Level + 10 m, based on a LAT map, and a Bathymetry and \nelevation GEBCO map. \n\n    Author: Mario.FuentesMonjaraz@deltares.nl\n'

### Define packages

In [25]:
import os
import dask
import xarray as xr
import pyproj
import rioxarray
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import re
import glob
from scipy.ndimage import binary_dilation

### Define functions

In [26]:
# Define a function to extract the coordinate part of the filename
def extract_coordinates(filename):
    match = re.search(r'gebco_2024_(n-?\d+\.\d+_s-?\d+\.\d+_w-?\d+\.\d+_e-?\d+\.\d+)', filename)
    if match:
        return match.group(1)
    else:
        return None

def assign_projection(ds, epsg=None):

    if not epsg == None:
        proj = pyproj.CRS.from_epsg(int(epsg))
    else:
        proj = pyproj.CRS.from_epsg(int(ds.crs.values.tolist()))

    print(proj,"projection was assigned to the dataset attributes")
    ds.attrs['crs'] = proj
    return ds

def print_ds_properties(rds,epsg=None):
    # Print the grid size
    print("Grid size:", rds.rio.resolution())

    #Print null data
    print("no data:", rds.rio.nodata)

    # Print the projection information
    if rds.rio.crs == None:
        print("There is no projection")
        proj = pyproj.CRS.from_epsg(epsg)
        rds.attrs['crs'] = proj
        rds.rio.set_crs(proj, inplace=True)
    else:
        print("There is projection available")
    
    print("Projection EPSG code is:", rds.rio.crs, "\n")
    return

def change_resolution(ds, new_resolution):
    # Reproject the rioxarray object to the new resolution
    reprojected_ds = ds.rio.reproject(ds.rio.crs, resolution=new_resolution, resampling="bilinear")
    return reprojected_ds

def match_resolution(rds, rds_source):
    # Reproject the rioxarray object to the new resolution
    reprojected_ds = rds.rio.reproject(rds_source.rio.crs, resolution=rds_source.rio.resolution(), resampling= rioxarray.enums.Resampling.bilinear)
    return reprojected_ds

def redefine_null_for_nan(ds, new_null_value):
    # Replace NaN values with the new null value
    ds.values[np.isnan(ds.values)] = new_null_value
    # ds.rio.update({'nodata': new_null_value})
    return ds

def create_gdf_from_geojson_files(input_aoi_data):
    geojson_files = []

    for filename in os.listdir(input_aoi_data):
        if filename.endswith(".geojson"):
            gdf = gpd.read_file(os.path.join(input_aoi_data,filename))
            aoi_id = filename.split('_')
            aoi_id = aoi_id[-1].split('.')[0]
            gdf.insert(1, "aoi", aoi_id)
            geojson_files.append((gdf))

    aoi_gdf = gpd.GeoDataFrame(pd.concat(geojson_files, ignore_index=True)).drop(columns=["id"])
    return aoi_gdf

def clip_raster(rds, geometry):
    rds_clipped = rds.rio.clip(geometry)
    return rds_clipped

def clip_raster_with_gdf(rds, gdf):
    rds_clipped_list = []
    for index, row in gdf.iterrows():
        aoi = row["aoi"]
        try:
            geometry = gdf.iloc[index:index+1].geometry
            rds_clipped = rds.rio.clip(geometry)
            rds_clipped_list.append(rds_clipped)
            print(f"Successful processing row {index} {aoi}")
        except Exception as e:
            print(f"Error processing row {index} {aoi}: {e}")
            rds_clipped_list.append("NaN")
            continue
    print("\n")
    return rds_clipped_list 

def apply_lat_mask(depth, lat):

    binary_mask = lat > depth
    depth_lat = depth.where(~binary_mask, other=np.nan)
    return depth_lat 

def apply_hat_mask(depth_lat, hat_rds):

    binary_mask =  depth_lat > hat_rds 
    depth_lat_hat = depth_lat.where(~binary_mask, other=np.nan)
    return depth_lat_hat

def apply_lat_mask_0(depth, lat):

    binary_mask = lat > depth
    depth_lat = depth.where(~binary_mask, other=0.0)
    return depth_lat 

def apply_hat_mask_0(depth_lat, hat_rds):

    binary_mask =  depth_lat > hat_rds 
    depth_lat_hat = depth_lat.where(~binary_mask, other=0.0)
    return depth_lat_hat

def apply_lat_mask_in(depth, lat):

    binary_mask = lat > depth
    depth_lat = depth.where(~binary_mask, other=np.nan)
    return depth_lat 

def apply_lat_mask_ex(depth, lat):

    binary_mask = lat <= depth
    depth_lat = depth.where(~binary_mask, other=np.nan)
    return depth_lat 

def apply_hat_mask_in(depth_lat, hat_rds):

    binary_mask =  depth_lat > hat_rds 
    depth_lat_hat = depth_lat.where(~binary_mask, other=np.nan)
    return depth_lat_hat

def apply_hat_mask_ex(depth_lat, hat_rds):

    binary_mask =  depth_lat <= hat_rds 
    depth_lat_hat = depth_lat.where(~binary_mask, other=np.nan)
    return depth_lat_hat

def get_coastline(lw_rds, cathegory):
    mask = cathegory
    masked_data = lw_rds.where(mask, other=np.nan)
    return masked_data

def save_mask(depth_lat_hat, name, output_data_path, coordinates):
    # Assign null values to the created mask
    depth_lat_hat_clean = xr.where((depth_lat_hat.isnull()) | (depth_lat_hat == 9999.0), np.nan, depth_lat_hat)
    depth_lat_hat_ones  = xr.where(depth_lat_hat_clean.isnull(), np.nan, depth_lat_hat_clean / depth_lat_hat_clean)

    # depth_lat.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat.tif"), crs=f"EPSG:{4326}")
    # depth_lat_hat.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_{name}.tif"), crs=f"EPSG:{4326}")
    # depth_lat_hat_ones.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_{name}_ones.tif"), crs=f"EPSG:{4326}")

    return depth_lat_hat_ones

def get_buffer_pixels(raster1, raster2, nameid, output_data_path, coordinates):
    # Load the two raster datasets
    raster1 = raster1.isel(band=0)
    raster2 = raster2.isel(band=0)

    # Ensure the datasets have the necessary rioxarray attributes and CRS
    raster1 = raster1.rio.write_crs("EPSG:4326")
    raster2 = raster2.rio.write_crs("EPSG:4326")

    # # Reproject raster2 to match the grid of raster1 if they do not have the same CRS and resolution
    raster2_reprojected = raster2.rio.reproject_match(raster1)

    # Define the proximity threshold in pixels (e.g., 1 pixel radius)
    proximity_threshold = 1

    # Create a binary mask where raster2 has valid data
    raster2_mask = ~np.isnan(raster2_reprojected)

    # Check the dimensions of the mask
    print(f"raster2_mask shape: {raster2_mask.shape}")

    # Create a structure element for dilation
    structure = np.ones((2 * proximity_threshold + 1, 2 * proximity_threshold + 1))
    print(f"structure shape: {structure.shape}")

    # Use binary dilation to identify proximity pixels
    proximity_mask = binary_dilation(raster2_mask, structure=structure)

    # Convert boolean mask to integer mask (0 and 1)
    proximity_mask_int = proximity_mask.astype(int)

    # Create a new DataArray for proximity pixels
    proximity_pixels = xr.DataArray(
        proximity_mask_int,
        dims=raster1.dims,
        coords=raster1.coords,
        attrs=raster1.attrs
    )

    # Optional: Save the proximity pixels to a new raster file
    # proximity_pixels.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_proximity_pixels_{nameid}.tif'))

    # For visualization, overlay the proximity mask on raster1
    raster1_with_proximity = raster1.copy()
    raster1_with_proximity.values[proximity_mask] = np.nan  # Masking proximity pixels

    # Save the overlaid result if needed
    # raster1_with_proximity.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_lat_in_with_proximity_{nameid}.tif'))

    return proximity_pixels, raster1_with_proximity

### Define packages paths

In [27]:
repository_path = os.path.dirname(os.getcwd())
input_data_path = r"D:\Proyectos2024\Copernicus\Repository\Copernicus\Data" #This folder has to exist with alll the LAT and HAT data that can be downloaded here https://nx1512.your-storageshare.de/s/Xzc6BgHCZ37KbZz
input_aoi_data = r"p:\11209821-cmems-global-sdb\00_miscellaneous\AOIs"
output_data_path = os.path.join(repository_path,"AOI_results_2024")

if not os.path.exists(output_data_path):
    print("Output data path does not exist. Creating directory...")
    os.makedirs(output_data_path)
    print("Output data path created:", output_data_path)
else:
    print("Input data path already exists:", output_data_path)

Input data path already exists: d:\Proyectos2024\Copernicus\Repository\Copernicus\Repository\eo-bathymetry\notebooks\AOI_results_2024


In [30]:
# Get HAT and LAT files
file_paths = glob.glob(os.path.join(input_data_path, 'gebco_2024*.tif'))
filenames = [os.path.basename(document) for document in file_paths]

# Get area of coverage of each gebco file
coordinates_list = [extract_coordinates(filename) for filename in filenames]
coordinates_list = set(coordinates_list)
coordinates_list = list(coordinates_list)

# Reduce the datasets for testing
# coordinates_list = ['n0.0_s-90.0_w90.0_e180.0', 'n0.0_s-90.0_w-180.0_e-90.0','n90.0_s0.0_w90.0_e180.0']
# coordinates_list.remove('n90.0_s0.0_w0.0_e90.0')
# coordinates_list.remove('n0.0_s-90.0_w90.0_e180.0')
# coordinates_list.remove('n0.0_s-90.0_w-180.0_e-90.0')
# coordinates_list.remove('n90.0_s0.0_w90.0_e180.0')
coordinates_list = ['n0.0_s-90.0_w90.0_e180.0']
coordinates_list

['n0.0_s-90.0_w90.0_e180.0']

In [35]:
for coordinates in coordinates_list:

    # Read gebco, lat, and hat files
    depthmsl_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_depthmsl.nc'), chunks={'y': 100, 'x':100})
    lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'y': 100, 'x':100})
    hat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'y': 100, 'x':100})
    lw_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'LandWater15ARC.nc'))

    print_ds_properties(depthmsl_rds)
    print_ds_properties(lat_rds)
    print_ds_properties(hat_rds)
    print_ds_properties(lw_rds, 4326)

Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666501832161, -0.004166666690214726)
no data: 255
There is no projection
Projection EPSG code is: EPSG:4326 



In [36]:
depthmsl_rds 

<xarray.DataArray 'DEPTH' (band: 1, y: 21600, x: 21600)> Size: 2GB
dask.array<open_rasterio-f2528aeed7ab9343597e88e801f43cc3<this-array>, shape=(1, 21600, 21600), dtype=float32, chunksize=(1, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int32 4B 1
  * x        (x) float64 173kB 90.0 90.01 90.01 90.01 ... 180.0 180.0 180.0
  * y        (y) float64 173kB -0.002083 -0.00625 -0.01042 ... -89.99 -90.0
    crs      int32 4B 0
Attributes:
    lat#long_name:      longitude
    lat#standard_name:  longitude
    lat#units:          degrees_east
    lon#long_name:      latitude
    lon#standard_name:  latitude
    lon#units:          degrees_north
    title:              Depth relative to MSL for GEBCO 2023 bathymetry
    long_name:          DEPTH
    _FillValue:         9999.0
    scale_factor:       1.0
    add_offset:         0.0

In [38]:
depthmsl_rds 

<xarray.DataArray 'elevation' (band: 1, y: 21600, x: 21600)> Size: 933MB
dask.array<open_rasterio-fe8e711d8d0477b9b7ed0a8559d25975<this-array>, shape=(1, 21600, 21600), dtype=int16, chunksize=(1, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int32 4B 1
  * x        (x) float64 173kB 90.0 90.01 90.01 90.01 ... 180.0 180.0 180.0
  * y        (y) float64 173kB -0.002083 -0.00625 -0.01042 ... -89.99 -90.0
    crs      int32 4B 0
Attributes: (12/63)
    AREA_OR_POINT:                   Area
    lat#axis:                        Y
    lat#long_name:                   latitude
    lat#sdn_parameter_name:          Latitude north
    lat#sdn_parameter_urn:           SDN:P01::ALATZZ01
    lat#sdn_uom_name:                Degrees north
    ...                              ...
    sdn_uom_urn:                     SDN:P06::ULAA
    standard_name:                   height_above_mean_sea_level
    units:                           m
    _FillValue:                      -32767
    scale_factor:                    1.0
    add_offset:                      0.0

In [37]:
for coordinates in coordinates_list:

    # Read gebco, lat, and hat files
    depthmsl_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2024_{coordinates}.tif'), chunks={'y': 100, 'x':100})
    lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'y': 100, 'x':100})
    hat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'y': 100, 'x':100})
    lw_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'LandWater15ARC.nc'))

    print_ds_properties(depthmsl_rds)
    print_ds_properties(lat_rds)
    print_ds_properties(hat_rds)
    print_ds_properties(lw_rds, 4326)

Grid size: (0.004166666666666668, -0.004166666666666667)
no data: -32767
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

Grid size: (0.004166666501832161, -0.004166666690214726)
no data: 255
There is no projection
Projection EPSG code is: EPSG:4326 



In [ ]:
for coordinates in coordinates_list:

    # Read gebco, lat, and hat files
    depthmsl_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_depthmsl.nc'), chunks={'y': 100, 'x':100})
    lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'y': 100, 'x':100})
    hat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'y': 100, 'x':100})
    lw_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'LandWater15ARC.nc'))

    print_ds_properties(depthmsl_rds)
    print_ds_properties(lat_rds)
    print_ds_properties(hat_rds)
    print_ds_properties(lw_rds, 4326)

    # Harmonization of layers
    # lw_rds = lw_rds.rio.reproject_match(depthmsl_rds)
    # minx, miny, maxx, maxy = depthmsl_rds.rio.bounds()
    # lw_rds = lw_rds.rio.clip_box(minx, miny, maxx, maxy)

    # depthmsl_rds = depthmsl_rds.rio.set_nodata(np.nan)
    # lat_rds      = lat_rds.rio.set_nodata(np.nan)
    # hat_rds      = hat_rds.rio.set_nodata(np.nan)
    # lw_rds       = lat_rds.rio.set_nodata(np.nan)

    # print_ds_properties(depthmsl_rds)
    # print_ds_properties(lat_rds)
    # print_ds_properties(hat_rds)
    # print_ds_properties(lw_rds)

    # depthmsl_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_depthmsl_rds.tif'), driver='GTiff', compress='lzw')
    # lat_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_lat_rds.tif'), driver='GTiff', compress='lzw')
    # hat_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_hat_rds.tif'), driver='GTiff', compress='lzw')
    # lw_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_lw_rds.tif'), driver='GTiff', compress='lzw')

    # depthmsl_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_depthmsl_rds.tif'))
    # lat_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_lat_rds.tif'))
    # hat_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_hat_rds.tif'))
    # lw_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_lw_rds.tif'))
    
    # print_ds_properties(depthmsl_rds)
    # print_ds_properties(lat_rds)
    # print_ds_properties(hat_rds)
    # print_ds_properties(lw_rds)

    # Get inclusive and exclusive areas
    depth_lat_in = apply_lat_mask_in(depthmsl_rds, lat_rds)
    depth_lat_ex = apply_lat_mask_ex(depthmsl_rds, lat_rds)
    depth_hat_in = apply_hat_mask_in(depthmsl_rds, hat_rds)
    depth_hat_ex = apply_hat_mask_ex(depthmsl_rds, hat_rds)
    depth_lat     = apply_lat_mask(depthmsl_rds, lat_rds)
    depth_lat_hat = apply_hat_mask(depth_lat, hat_rds)

    lat_in = save_mask(depth_lat_in, 'lat_in', output_data_path, coordinates)
    lat_ex = save_mask(depth_lat_ex, 'lat_ex',output_data_path, coordinates)
    hat_in = save_mask(depth_hat_in, 'hat_in',output_data_path, coordinates)
    hat_ex = save_mask(depth_hat_ex, 'hat_ex',output_data_path, coordinates)
    lat_hat = save_mask(depth_lat_hat, 'lat_hat',output_data_path, coordinates)
    lat_hat.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif"), crs=f"EPSG:{4326}")
    
    # lat_in = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_in_ones.tif"))
    # lat_ex = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_ex_ones.tif"))
    # hat_in = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_in_ones.tif"))
    # hat_ex = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_ex_ones.tif"))
    # lat_hat = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif"))

    # Get filling pixels
    hat_ex_mask        = hat_ex.where(hat_ex == 1.0, other=0)
    landwater_mask     = lw_rds.where(lw_rds == 2.0, other=0)

    # hat_ex_mask.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_hat_ex_mask.tif"), crs=f"EPSG:{4326}")
    # landwater_mask.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_landwater_mask.tif"), crs=f"EPSG:{4326}")
    # hat_ex_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_hat_ex_mask.tif"))
    # landwater_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_landwater_mask.tif"))

    print_ds_properties(hat_ex_mask, 4326)
    print_ds_properties(landwater_mask, 4326)

    landwater_mask = landwater_mask.rio.reproject_match(hat_ex_mask)
    minx, miny, maxx, maxy = hat_ex_mask.rio.bounds()
    landwater_mask = landwater_mask.rio.clip_box(minx, miny, maxx, maxy)

    hat_ex_mask = hat_ex_mask.rio.set_nodata(np.nan)
    landwater_mask = landwater_mask.rio.set_nodata(np.nan)

    print_ds_properties(hat_ex_mask)
    print_ds_properties(landwater_mask)

    addition = hat_ex_mask + landwater_mask 
    addition = addition.where(addition != 0, np.nan)

    # addition.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"), crs=f"EPSG:{4326}")
    # addition = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"))

    buffer, raster1_with_proximity = get_buffer_pixels(addition, lat_ex, 'lat_ex', output_data_path, coordinates)

    coastline = addition.where(buffer==1.0, other=np.nan)

    # coastline.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_coastline.tif'))
    # coastline = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"))

    lat_hat_0 =  lat_hat.where(lat_hat==1.0, other=0.0)
    coastline_0 = coastline.where((coastline == 1.0) | (coastline == 2.0), other=0.0)

    # lat_hat_0.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_0.tif"), crs=f"EPSG:{4326}")
    # coastline_0.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_coastline_0.tif"), crs=f"EPSG:{4326}")

    result = lat_hat_0*3 + coastline_0
    result = result.where(result!=0.0, other=np.nan)
    result.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_result.tif'))

    # Open raster file using rasterio
    with rasterio.open(os.path.join(output_data_path,f"gebco_2023_{coordinates}_result.tif")) as src:
        # Read raster data into numpy array
        raster_array = src.read(1)  # Assuming it's a single band raster, adjust if necessary
        # Extract transformation metadata
        transform = src.transform
        # Polygonize raster data
        polygons = list(shapes(raster_array, mask=None, transform=transform))
        # Convert polygons to Shapely geometries and record pixel values
        geometries_with_values = [(shape(polygon), value) for polygon, value in polygons]

    # Extract geometries and values into separate lists
    geometries = [geometry for geometry, value in geometries_with_values]
    values = [value for geometry, value in geometries_with_values]

    # Convert Shapely geometries and pixel values to GeoDataFrame
    geo_df = gpd.GeoDataFrame(geometry=geometries, data={'pixel_value': values})

    geo_df = geo_df[~np.isnan(geo_df['pixel_value'])]
    geo_df.reset_index(drop=True, inplace=True)

    # Define the EPSG code for the desired projection
    epsg_code = 4326  # For example, EPSG code for WGS 84

    # Assign the projection to the GeoDataFrame
    geo_df.crs = f"EPSG:{epsg_code}"

    # Save GeoDataFrame to file
    # geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat_hat.shp"))
    geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_result.geojson"), driver="GeoJSON", crs=f"EPSG:{epsg_code}")


In [ ]:
for coordinates in coordinates_list:

    # Read gebco, lat, and hat files
    depthmsl_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_depthmsl.nc'), chunks={'y': 100, 'x':100})
    lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'y': 100, 'x':100})
    hat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'y': 100, 'x':100})
    lw_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'LandWater15ARC.nc'))

    print_ds_properties(depthmsl_rds)
    print_ds_properties(lat_rds)
    print_ds_properties(hat_rds)
    print_ds_properties(lw_rds, 4326)

    # Harmonization of layers
    # lw_rds = lw_rds.rio.reproject_match(depthmsl_rds)
    # minx, miny, maxx, maxy = depthmsl_rds.rio.bounds()
    # lw_rds = lw_rds.rio.clip_box(minx, miny, maxx, maxy)

    # depthmsl_rds = depthmsl_rds.rio.set_nodata(np.nan)
    # lat_rds      = lat_rds.rio.set_nodata(np.nan)
    # hat_rds      = hat_rds.rio.set_nodata(np.nan)
    # lw_rds       = lat_rds.rio.set_nodata(np.nan)

    # print_ds_properties(depthmsl_rds)
    # print_ds_properties(lat_rds)
    # print_ds_properties(hat_rds)
    # print_ds_properties(lw_rds)

    # depthmsl_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_depthmsl_rds.tif'), driver='GTiff', compress='lzw')
    # lat_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_lat_rds.tif'), driver='GTiff', compress='lzw')
    # hat_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_hat_rds.tif'), driver='GTiff', compress='lzw')
    # lw_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_lw_rds.tif'), driver='GTiff', compress='lzw')

    # depthmsl_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_depthmsl_rds.tif'))
    # lat_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_lat_rds.tif'))
    # hat_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_hat_rds.tif'))
    # lw_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_lw_rds.tif'))
    
    # print_ds_properties(depthmsl_rds)
    # print_ds_properties(lat_rds)
    # print_ds_properties(hat_rds)
    # print_ds_properties(lw_rds)

    # Get inclusive and exclusive areas
    depth_lat_in = apply_lat_mask_in(depthmsl_rds, lat_rds)
    depth_lat_ex = apply_lat_mask_ex(depthmsl_rds, lat_rds)
    depth_hat_in = apply_hat_mask_in(depthmsl_rds, hat_rds)
    depth_hat_ex = apply_hat_mask_ex(depthmsl_rds, hat_rds)
    depth_lat     = apply_lat_mask(depthmsl_rds, lat_rds)
    depth_lat_hat = apply_hat_mask(depth_lat, hat_rds)

    lat_in = save_mask(depth_lat_in, 'lat_in', output_data_path, coordinates)
    lat_ex = save_mask(depth_lat_ex, 'lat_ex',output_data_path, coordinates)
    hat_in = save_mask(depth_hat_in, 'hat_in',output_data_path, coordinates)
    hat_ex = save_mask(depth_hat_ex, 'hat_ex',output_data_path, coordinates)
    lat_hat = save_mask(depth_lat_hat, 'lat_hat',output_data_path, coordinates)
    lat_hat.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif"), crs=f"EPSG:{4326}")
    
    # lat_in = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_in_ones.tif"))
    # lat_ex = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_ex_ones.tif"))
    # hat_in = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_in_ones.tif"))
    # hat_ex = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_ex_ones.tif"))
    # lat_hat = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif"))

    # Get filling pixels
    hat_ex_mask        = hat_ex.where(hat_ex == 1.0, other=0)
    landwater_mask     = lw_rds.where(lw_rds == 2.0, other=0)

    # hat_ex_mask.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_hat_ex_mask.tif"), crs=f"EPSG:{4326}")
    # landwater_mask.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_landwater_mask.tif"), crs=f"EPSG:{4326}")
    # hat_ex_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_hat_ex_mask.tif"))
    # landwater_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_landwater_mask.tif"))

    print_ds_properties(hat_ex_mask, 4326)
    print_ds_properties(landwater_mask, 4326)

    landwater_mask = landwater_mask.rio.reproject_match(hat_ex_mask)
    minx, miny, maxx, maxy = hat_ex_mask.rio.bounds()
    landwater_mask = landwater_mask.rio.clip_box(minx, miny, maxx, maxy)

    hat_ex_mask = hat_ex_mask.rio.set_nodata(np.nan)
    landwater_mask = landwater_mask.rio.set_nodata(np.nan)

    print_ds_properties(hat_ex_mask)
    print_ds_properties(landwater_mask)

    addition = hat_ex_mask + landwater_mask 
    addition = addition.where(addition != 0, np.nan)

    # addition.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"), crs=f"EPSG:{4326}")
    # addition = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"))

    buffer, raster1_with_proximity = get_buffer_pixels(addition, lat_ex, 'lat_ex', output_data_path, coordinates)

    coastline = addition.where(buffer==1.0, other=np.nan)

    # coastline.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_coastline.tif'))
    # coastline = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"))

    lat_hat_0 =  lat_hat.where(lat_hat==1.0, other=0.0)
    coastline_0 = coastline.where((coastline == 1.0) | (coastline == 2.0), other=0.0)

    # lat_hat_0.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_0.tif"), crs=f"EPSG:{4326}")
    # coastline_0.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_coastline_0.tif"), crs=f"EPSG:{4326}")

    result = lat_hat_0*3 + coastline_0
    result = result.where(result!=0.0, other=np.nan)
    result.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_result.tif'))

    # Open raster file using rasterio
    with rasterio.open(os.path.join(output_data_path,f"gebco_2023_{coordinates}_result.tif")) as src:
        # Read raster data into numpy array
        raster_array = src.read(1)  # Assuming it's a single band raster, adjust if necessary
        # Extract transformation metadata
        transform = src.transform
        # Polygonize raster data
        polygons = list(shapes(raster_array, mask=None, transform=transform))
        # Convert polygons to Shapely geometries and record pixel values
        geometries_with_values = [(shape(polygon), value) for polygon, value in polygons]

    # Extract geometries and values into separate lists
    geometries = [geometry for geometry, value in geometries_with_values]
    values = [value for geometry, value in geometries_with_values]

    # Convert Shapely geometries and pixel values to GeoDataFrame
    geo_df = gpd.GeoDataFrame(geometry=geometries, data={'pixel_value': values})

    geo_df = geo_df[~np.isnan(geo_df['pixel_value'])]
    geo_df.reset_index(drop=True, inplace=True)

    # Define the EPSG code for the desired projection
    epsg_code = 4326  # For example, EPSG code for WGS 84

    # Assign the projection to the GeoDataFrame
    geo_df.crs = f"EPSG:{epsg_code}"

    # Save GeoDataFrame to file
    # geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat_hat.shp"))
    geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_result.geojson"), driver="GeoJSON", crs=f"EPSG:{epsg_code}")


In [8]:
for coordinates in coordinates_list:

    # # Read gebco, lat, and hat files
    # depthmsl_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_depthmsl.nc'), chunks={'y': 100, 'x':100})
    # lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'y': 100, 'x':100})
    # hat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'y': 100, 'x':100})
    # lw_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'LandWater15ARC.nc'))

    # print_ds_properties(depthmsl_rds)
    # print_ds_properties(lat_rds)
    # print_ds_properties(hat_rds)
    # print_ds_properties(lw_rds, 4326)

    # # Harmonization of layers
    # # lw_rds = lw_rds.rio.reproject_match(depthmsl_rds)
    # # minx, miny, maxx, maxy = depthmsl_rds.rio.bounds()
    # # lw_rds = lw_rds.rio.clip_box(minx, miny, maxx, maxy)

    # # depthmsl_rds = depthmsl_rds.rio.set_nodata(np.nan)
    # # lat_rds      = lat_rds.rio.set_nodata(np.nan)
    # # hat_rds      = hat_rds.rio.set_nodata(np.nan)
    # # lw_rds       = lat_rds.rio.set_nodata(np.nan)

    # # print_ds_properties(depthmsl_rds)
    # # print_ds_properties(lat_rds)
    # # print_ds_properties(hat_rds)
    # # print_ds_properties(lw_rds)

    # # depthmsl_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_depthmsl_rds.tif'), driver='GTiff', compress='lzw')
    # # lat_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_lat_rds.tif'), driver='GTiff', compress='lzw')
    # # hat_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_hat_rds.tif'), driver='GTiff', compress='lzw')
    # # lw_rds.rio.to_raster(os.path.join(output_data_path,f'{coordinates}_lw_rds.tif'), driver='GTiff', compress='lzw')

    # # depthmsl_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_depthmsl_rds.tif'))
    # # lat_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_lat_rds.tif'))
    # # hat_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_hat_rds.tif'))
    # # lw_rds = rioxarray.open_rasterio(os.path.join(output_data_path,f'{coordinates}_lw_rds.tif'))
    
    # # print_ds_properties(depthmsl_rds)
    # # print_ds_properties(lat_rds)
    # # print_ds_properties(hat_rds)
    # # print_ds_properties(lw_rds)

    # # Get inclusive and exclusive areas
    # depth_lat_in = apply_lat_mask_in(depthmsl_rds, lat_rds)
    # depth_lat_ex = apply_lat_mask_ex(depthmsl_rds, lat_rds)
    # depth_hat_in = apply_hat_mask_in(depthmsl_rds, hat_rds)
    # depth_hat_ex = apply_hat_mask_ex(depthmsl_rds, hat_rds)
    # depth_lat     = apply_lat_mask(depthmsl_rds, lat_rds)
    # depth_lat_hat = apply_hat_mask(depth_lat, hat_rds)

    # lat_in = save_mask(depth_lat_in, 'lat_in', output_data_path, coordinates)
    # lat_ex = save_mask(depth_lat_ex, 'lat_ex',output_data_path, coordinates)
    # hat_in = save_mask(depth_hat_in, 'hat_in',output_data_path, coordinates)
    # hat_ex = save_mask(depth_hat_ex, 'hat_ex',output_data_path, coordinates)
    # lat_hat = save_mask(depth_lat_hat, 'lat_hat',output_data_path, coordinates)
    # lat_hat.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif"), crs=f"EPSG:{4326}")
    
    # # lat_in = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_in_ones.tif"))
    # # lat_ex = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_ex_ones.tif"))
    # # hat_in = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_in_ones.tif"))
    # # hat_ex = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_ex_ones.tif"))
    # # lat_hat = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif"))

    # # Get filling pixels
    # hat_ex_mask        = hat_ex.where(hat_ex == 1.0, other=0)
    # landwater_mask     = lw_rds.where(lw_rds == 2.0, other=0)

    # # hat_ex_mask.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_hat_ex_mask.tif"), crs=f"EPSG:{4326}")
    # # landwater_mask.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_landwater_mask.tif"), crs=f"EPSG:{4326}")
    # # hat_ex_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_hat_ex_mask.tif"))
    # # landwater_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_landwater_mask.tif"))

    # print_ds_properties(hat_ex_mask, 4326)
    # print_ds_properties(landwater_mask, 4326)

    # landwater_mask = landwater_mask.rio.reproject_match(hat_ex_mask)
    # minx, miny, maxx, maxy = hat_ex_mask.rio.bounds()
    # landwater_mask = landwater_mask.rio.clip_box(minx, miny, maxx, maxy)

    # hat_ex_mask = hat_ex_mask.rio.set_nodata(np.nan)
    # landwater_mask = landwater_mask.rio.set_nodata(np.nan)

    # print_ds_properties(hat_ex_mask)
    # print_ds_properties(landwater_mask)

    # addition = hat_ex_mask + landwater_mask 
    # addition = addition.where(addition != 0, np.nan)

    # # addition.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"), crs=f"EPSG:{4326}")
    # # addition = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"))

    # buffer, raster1_with_proximity = get_buffer_pixels(addition, lat_ex, 'lat_ex', output_data_path, coordinates)

    # coastline = addition.where(buffer==1.0, other=np.nan)

    # # coastline.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_coastline.tif'))
    # # coastline = rioxarray.open_rasterio(os.path.join(output_data_path,f"gebco_2023_{coordinates}_addition.tif"))

    # lat_hat_0 =  lat_hat.where(lat_hat==1.0, other=0.0)
    # coastline_0 = coastline.where((coastline == 1.0) | (coastline == 2.0), other=0.0)

    # # lat_hat_0.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_0.tif"), crs=f"EPSG:{4326}")
    # # coastline_0.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_coastline_0.tif"), crs=f"EPSG:{4326}")

    # result = lat_hat_0*3 + coastline_0
    # result = result.where(result!=0.0, other=np.nan)
    # result.rio.to_raster(os.path.join(output_data_path,f'gebco_2023_{coordinates}_result.tif'))

    # Open raster file using rasterio
    with rasterio.open(os.path.join(output_data_path,f"gebco_2023_{coordinates}_result.tif")) as src:
        # Read raster data into numpy array
        raster_array = src.read(1)  # Assuming it's a single band raster, adjust if necessary
        # Extract transformation metadata
        transform = src.transform
        # Polygonize raster data
        polygons = list(shapes(raster_array, mask=None, transform=transform))
        # Convert polygons to Shapely geometries and record pixel values
        geometries_with_values = [(shape(polygon), value) for polygon, value in polygons]

    # Extract geometries and values into separate lists
    geometries = [geometry for geometry, value in geometries_with_values]
    values = [value for geometry, value in geometries_with_values]

    # Convert Shapely geometries and pixel values to GeoDataFrame
    geo_df = gpd.GeoDataFrame(geometry=geometries, data={'pixel_value': values})

    geo_df = geo_df[~np.isnan(geo_df['pixel_value'])]
    geo_df.reset_index(drop=True, inplace=True)

    # Define the EPSG code for the desired projection
    epsg_code = 4326  # For example, EPSG code for WGS 84

    # Assign the projection to the GeoDataFrame
    geo_df.crs = f"EPSG:{epsg_code}"

    # Save GeoDataFrame to file
    # geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat_hat.shp"))
    geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_result.geojson"), driver="GeoJSON", crs=f"EPSG:{epsg_code}")
